# Machine Learning on MCU - Ex3.2

This exercise will introduce you to Keras, one of the most popular frameworks used in Deep Learning.

You will train a simple multi-layer perceptron to predict Human Activity from Smartphone Accelerometer and Gyroscope Data.

We use a dataset of 3-axial accelerometer signals from an academic experiment on the UC Irvine Machine Learning Repository.

The dataset is downloaded in the code snippet below and you can you can also find the description of the dataset [here](https://archive.ics.uci.edu/dataset/341/smartphone+based+recognition+of+human+activities+and+postural+transitions) .

# Load the dataset

In [2]:
# If required, download the dataset
import requests
import os.path
import zipfile
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random

random.seed(7)

if (not os.path.isdir('./HAPT Data Set')):
    open('./HAPT Data Set.zip', 'wb').write(requests.get(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/00341/HAPT%20Data%20Set.zip", 
        allow_redirects=True).content)
    zipfile.ZipFile('./HAPT Data Set.zip', 'r').extractall('./HAPT Data Set')

Load the accelerometer and gyroscope data.
We read the feature names from features.txt and the activity labels from activity_labels.txt

In [37]:
with open('./HAPT Data Set/features.txt') as f:
    features = f.read().split()

print('There are {} features.'.format(len(features)))
    
with open('./HAPT Data Set/activity_labels.txt') as f:
    activity_labels = f.readlines()

activity_df = [x.split() for x in activity_labels]
print('There are {} activities.'.format(len(activity_df)))
pd.DataFrame(activity_df, columns = ['Activity_id', 'Activity_label'])

There are 561 features.
There are 12 activities.


,Activity_id,Activity_label
0,1,WALKING
1,2,WALKING_UPSTAIRS
2,3,WALKING_DOWNSTAIRS
3,4,SITTING
4,5,STANDING
5,6,LAYING
6,7,STAND_TO_SIT
7,8,SIT_TO_STAND
8,9,SIT_TO_LIE
9,10,LIE_TO_SIT


The data are pre-split into training and test sets. Let's load the features x and the labels y, and have a look at a few features.

In [38]:
X_train = pd.read_table('./HAPT Data Set/Train/X_train.txt',
             header = None, sep = " ", names = list(dict.fromkeys(features)))
X_train.iloc[:10, :10].head()

,,,,,,,,,,,,,,,,,,,,,,,,,,,,tBodyAcc-Mean-1,tBodyAcc-Mean-2,tBodyAcc-Mean-3,tBodyAcc-STD-1,tBodyAcc-STD-2,tBodyAcc-STD-3,tBodyAcc-Mad-1,tBodyAcc-Mad-2,tBodyAcc-Mad-3,tBodyAcc-Max-1
0.043580,-0.005970,-0.035054,-0.995381,-0.988366,-0.937382,-0.995007,-0.988816,-0.953325,-0.794796,-0.744893,-0.648447,0.841796,0.708440,0.651716,-0.975752,-0.999950,-0.999888,-0.998014,-0.993999,-0.991980,-0.970970,-0.547095,-0.700974,-0.622697,0.921884,-0.719483,0.342168,-0.161318,0.266049,-0.274351,0.267205,-0.020958,0.382610,-0.501748,0.512463,-0.206337,0.376778
0.039480,-0.002131,-0.029067,-0.998348,-0.982945,-0.971273,-0.998702,-0.983315,-0.974000,-0.802537,-0.736338,-0.712415,0.838758,0.708440,0.659340,-0.987427,-0.999993,-0.999826,-0.999411,-0.998918,-0.985482,-0.973481,-0.781973,-0.534604,-0.593165,0.607435,-0.266783,0.275882,0.200417,0.131266,-0.149017,0.292436,-0.192986,0.217496,-0.089175,0.059909,-0.236609,-0.012696
0.039978,-0.005153,-0.022651,-0.995482,-0.977314,-0.984760,-0.996415,-0.975835,-0.985973,-0.798477,-0.736338,-0.712415,0.834002,0.705008,0.674551,-0.988528,-0.999972,-0.999719,-0.999803,-0.996898,-0.976781,-0.986754,-0.688176,-0.520514,-0.593165,0.272262,-0.056424,0.322283,-0.273292,0.037180,-0.133612,0.332487,-0.240491,0.348733,-0.195409,0.229436,-0.316816,-0.123889
0.039785,-0.011809,-0.028916,-0.996194,-0.988569,-0.993256,-0.996994,-0.988526,-0.993135,-0.798477,-0.752778,-0.722186,0.834002,0.705008,0.673208,-0.990389,-0.999978,-0.999783,-0.999815,-0.996949,-0.989437,-0.992440,-0.715103,-0.860988,-0.916429,0.062816,0.082940,0.200566,-0.378262,0.090063,-0.209264,0.316530,-0.090862,0.396383,-0.353643,0.503754,-0.490389,-0.304759
0.038758,-0.002289,-0.023863,-0.998241,-0.986774,-0.993115,-0.998216,-0.986479,-0.993825,-0.801982,-0.746505,-0.717858,0.838581,0.705854,0.673208,-0.995057,-0.999992,-0.999882,-0.999908,-0.997772,-0.987726,-0.995109,-0.836774,-0.589200,-0.773771,0.312105,-0.095254,0.194399,-0.007998,0.266740,-0.318965,0.409731,-0.224589,0.520354,-0.319167,0.234376,-0.102650,-0.154974


In [39]:
y_train = pd.read_table('./HAPT Data Set/Train/y_train.txt',
             header = None, sep = " ", names = ['Activity_id'])
y_train.head()

,Activity_id
0,5
1,5
2,5
3,5
4,5


In [40]:
X_test = pd.read_table('./HAPT Data Set/Test/X_test.txt',
             header = None, sep = " ", names = list(dict.fromkeys(features)))
y_test = pd.read_table('./HAPT Data Set/Test/y_test.txt',
             header = None, sep = " ", names = ['Activity_id'])

## Predict Human Activity

In [41]:
# Import the necessary packages
from sklearn.model_selection import train_test_split

# Note: use Tensor Flow backend for Keras as suggested on keras.io:
# "At this time, we recommend that Keras users who use multi-backend Keras with the TensorFlow backend switch to tf.keras in TensorFlow 2.0. tf.keras is better maintained and has better integration with TensorFlow features (eager execution, distribution support and other)."
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, models, regularizers

In [42]:
# Load the accelerometer and gyroscope data.

# ?

In [43]:
# Split the dataset into training, validation, and testing sets. (You can take example from ex 1 and 2.)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size =0.1,random_state=42)


In [50]:
X_train.shape

(6990, 533)

In [45]:
# Note: Keras takes the classes from 0, i.e. if you have classes [1, 2, 3] (as in this case), it will give you an error.
# To avoid the error, shift the class labels by -1, i.e. from [1, 2, 3] to [0, 1, 2]
y_val['Activity_id'] -= 1
y_test['Activity_id'] -= 1
y_train['Activity_id'] -= 1

In [47]:
max(y_val.values)

array([11], dtype=int64)

In [52]:
# Define batchsize and number of epochs
outputShape = (12)
bacthSize = 10
epochs = 5


# Declare the sequential model and design your multi-layer perceptron
model = models.Sequential()
model.add(layers.Dense(128,activation='relu'))
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(32,activation='relu'))
model.add(layers.Dense(outputShape,activation='softmax'))

# Compile your model
model.compile(loss='sparse_categorical_crossentropy',
optimizer=keras.optimizers.Adam(),
metrics=['accuracy'])

# Train your model
model.fit(X_train,y_train,bacthSize,epochs)


Epoch 1/5
699/699 [==============================] - 5s 6ms/step - loss: 0.5416 - accuracy: 0.8039
Epoch 2/5
699/699 [==============================] - 4s 5ms/step - loss: 0.2466 - accuracy: 0.9019
Epoch 3/5
699/699 [==============================] - 4s 6ms/step - loss: 0.1803 - accuracy: 0.9273
Epoch 4/5
699/699 [==============================] - 4s 5ms/step - loss: 0.1612 - accuracy: 0.9342
Epoch 5/5
699/699 [==============================] - 5s 7ms/step - loss: 0.1403 - accuracy: 0.9418


In [53]:
# You can use model.summary() to get an overview of your model


model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (10, 128)                 68352     
                                                                 
 dense_13 (Dense)            (10, 64)                  8256      
                                                                 
 dense_14 (Dense)            (10, 32)                  2080      
                                                                 
 dense_15 (Dense)            (10, 12)                  396       
                                                                 
Total params: 79,084
Trainable params: 79,084
Non-trainable params: 0
_________________________________________________________________


In [56]:
# Evaluate your model
score = model.evaluate(X_test,y_test)

99/99 [==============================] - 1s 4ms/step - loss: 0.3247 - accuracy: 0.8836


In [57]:
# Save your model once you are satisfied with it

model.save('ex_3_weights.keras')

In [58]:
# Load your saved model and test on the test data.

model = keras.models.load_model('ex_3_weights.keras')